<a href="https://colab.research.google.com/github/Dpgofast/DS-Unit-2-Sprint-3-Advanced-Regression/blob/master/Dakota_DS1_LS_DS1_231_Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lambda School Data Science - Logistic Regression

Logistic regression is the baseline for classification models, as well as a handy way to predict probabilities (since those too live in the unit interval). While relatively simple, it is also the foundation for more sophisticated classification techniques such as neural networks (many of which can effectively be thought of as networks of logistic models).

## Lecture - Where Linear goes Wrong
### Return of the Titanic 🚢

You've likely already explored the rich dataset that is the Titanic - let's use regression and try to predict survival with it. The data is [available from Kaggle](https://www.kaggle.com/c/titanic/data), so we'll also play a bit with [the Kaggle API](https://github.com/Kaggle/kaggle-api).

In [5]:
!pip install kaggle

In [6]:
# Note - you'll also have to sign up for Kaggle and authorize the API
# https://github.com/Kaggle/kaggle-api#api-credentials

# This essentially means uploading a kaggle.json file
# For Colab we can have it in Google Drive
from google.colab import drive
drive.mount('/content/drive')
%env KAGGLE_CONFIG_DIR=/content/drive/My Drive/

# You also have to join the Titanic competition to have access to the data
!kaggle competitions download -c titanic

KeyboardInterrupt: ignored

In [0]:
# How would we try to do this with linear regression?
import pandas as pd

train_df = pd.read_csv('train.csv').dropna()
test_df = pd.read_csv('test.csv').dropna()  # Unlabeled, for Kaggle submission

train_df.head()

In [0]:
train_df.describe()

In [0]:
from sklearn.linear_model import LinearRegression

X = train_df[['Pclass', 'Age', 'Fare']]
y = train_df.Survived

linear_reg = LinearRegression().fit(X, y)
linear_reg.score(X, y)

In [0]:
linear_reg.predict(test_df[['Pclass', 'Age', 'Fare']])

In [0]:
linear_reg.coef_

In [0]:
import numpy as np

test_case = np.array([[1, 5, 500]])  # Rich 5-year old in first class
linear_reg.predict(test_case)

In [0]:
np.dot(linear_reg.coef_, test_case.reshape(-1, 1)) + linear_reg.intercept_

In [0]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression().fit(X, y)
log_reg.score(X, y)

In [0]:
log_reg.predict(test_df[['Pclass', 'Age', 'Fare']])

In [0]:
log_reg.predict(test_case)[0]

In [0]:
help(log_reg.predict)

In [0]:
log_reg.predict_proba(test_case)[0]

In [0]:
# What's the math?
log_reg.coef_

In [0]:
log_reg.intercept_

In [0]:
# The logistic sigmoid "squishing" function, implemented to accept numpy arrays
def sigmoid(x):
  return 1 / (1 + np.e**(-x))

In [0]:
sigmoid(log_reg.intercept_ + np.dot(log_reg.coef_, np.transpose(test_case)))

So, clearly a more appropriate model in this situation! For more on the math, [see this Wikipedia example](https://en.wikipedia.org/wiki/Logistic_regression#Probability_of_passing_an_exam_versus_hours_of_study).

For live - let's tackle [another classification dataset on absenteeism](http://archive.ics.uci.edu/ml/datasets/Absenteeism+at+work) - it has 21 classes, but remember, scikit-learn LogisticRegression automatically handles more than two classes. How? By essentially treating each label as different (1) from some base class (0).

## Assignment - real-world classification

We're going to check out a larger dataset - the [FMA Free Music Archive data](https://github.com/mdeff/fma). It has a selection of CSVs with metadata and calculated audio features that you can load and try to use to classify genre of tracks. To get you started:

In [0]:
# Live - let's try absenteeism!
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
import numpy as np 
from sklearn.preprocessing import OneHotEncoder 


In [4]:
!wget https://os.unil.cloud.switch.ch/fma/fma_metadata.zip
!unzip fma_metadata.zip

--2019-01-22 15:37:36--  https://os.unil.cloud.switch.ch/fma/fma_metadata.zip
Resolving os.unil.cloud.switch.ch (os.unil.cloud.switch.ch)... 86.119.28.13, 2001:620:5ca1:2ff::ce53
Connecting to os.unil.cloud.switch.ch (os.unil.cloud.switch.ch)|86.119.28.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358412441 (342M) [application/zip]
Saving to: ‘fma_metadata.zip.1’

fma_metadata.zip.1  100%[===================>] 341.81M  24.5MB/s    in 20s     

2019-01-22 15:37:56 (17.1 MB/s) - ‘fma_metadata.zip.1’ saved [358412441/358412441]

Archive:  fma_metadata.zip
replace fma_metadata/README.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace fma_metadata/checksums? [y]es, [n]o, [A]ll, [N]one, [r]ename: no
replace fma_metadata/not_found.pickle? [y]es, [n]o, [A]ll, [N]one, [r]ename: no
replace fma_metadata/raw_genres.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: no
replace fma_metadata/raw_albums.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: no
replace fma_metadata/raw_art

In [3]:
tracks = pd.read_csv('fma_metadata/tracks.csv', header = None)
xtracks=tracks
xtracks.columns = tracks.iloc[1]
xtracks = xtracks.reindex(xtracks.index.drop([0,1,2]))
fix_trax = xtracks
fix_trax.head()
# goal is y=genre

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1,5,6,8,12,18,20,21,22,24,33,34,38,39,44,47,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


1,nan,comments,date_created,date_released,engineer,favorites,id,information,listens,producer,...,information,interest,language_code,license,listens,lyricist,number,publisher,tags,title
3,2,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,...,NaN,4656,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1293,NaN,3,NaN,[],Food
4,3,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,...,NaN,1470,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,514,NaN,4,NaN,[],Electric Ave
5,5,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,...,NaN,1933,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1151,NaN,6,NaN,[],This World
6,10,0,2008-11-26 01:45:08,2008-02-06 00:00:00,NaN,4,6,NaN,47632,NaN,...,NaN,54881,en,Attribution-NonCommercial-NoDerivatives (aka M...,50135,NaN,1,NaN,[],Freeway
7,20,0,2008-11-26 01:45:05,2009-01-06 00:00:00,NaN,2,4,"<p> ""spiritual songs"" from Nicky Cook</p>",2710,NaN,...,NaN,978,en,Attribution-NonCommercial-NoDerivatives (aka M...,361,NaN,3,NaN,[],Spiritual Level


In [4]:
fix_trax.shape

(106574, 53)

In [0]:
pd.set_option('display.max_columns', None)  # Unlimited columns
fix_trax.head()
pd.options.mode.use_inf_as_na = True

In [6]:
fix_trax.isna().sum()


1
NaN                       0
comments                  0
date_created           3529
date_released         36280
engineer              91279
favorites                 0
id                        0
information           23425
listens                   0
producer              88514
tags                      0
title                  1025
tracks                    0
type                   6508
active_year_begin     83863
active_year_end      101199
associated_labels     92303
bio                   35418
comments                  0
date_created            856
favorites                 0
id                        0
latitude              62030
location              36364
longitude             62030
members               59725
name                      0
related_projects      93422
tags                      0
website               27318
wikipedia_page       100993
split                     0
subset                    0
bit_rate                  0
comments                  0
composer          

In [33]:
# lets make a list  of columns to drop.
drops = ['date_created ','date_released','engineer','information','producer',
         'type','active_year_begin','active_year_end','associated_labels',
         'bio','latitude','location','longitude','members','related_projects',
         'website','wikipedia_page','composer','date_recorded',
         'information','language_code','lyricist','publisher','license',
         'title','date_created','nan','tags','genres','comments','listens',
         'favorites','id','name','subset','genres_all']
drop = fix_trax.drop(columns = drops, axis = 'columns')
drop.isna().sum().sum()

56976

In [34]:
drop.genre_top

3              Hip-Hop
4              Hip-Hop
5              Hip-Hop
6                  Pop
7                  NaN
8                  NaN
9                  NaN
10                 NaN
11                 NaN
12             Hip-Hop
13                Rock
14                Rock
15        Experimental
16        Experimental
17                Folk
18                Folk
19                Folk
20                Folk
21                Jazz
22                Jazz
23                Jazz
24                Jazz
25        Experimental
26        Experimental
27        Experimental
28                Rock
29                Rock
30                Rock
31                Rock
32                Rock
              ...     
106547    Instrumental
106548    Instrumental
106549    Instrumental
106550    Instrumental
106551    Instrumental
106552    Instrumental
106553    Instrumental
106554    Instrumental
106555            Folk
106556            Folk
106557            Folk
106558            Folk
106559     

In [76]:
drop = drop.dropna()
train= drop[drop['split']=='training']
test  = drop[drop['split']=='test']
train1= train.drop(columns = 'split', axis = 1 )
test1 = test.drop(columns = 'split', axis = 1)
xtest = test1.drop(columns = 'genre_top', axis = 'columns')
ytest = test1['genre_top']
xtrain = train1.drop(columns ='genre_top', axis = 'columns' )
ytrain = train1['genre_top']
xtrain.head()

1,nan,tracks,bit_rate,duration,interest,number
3,2,7,256000,168,4656,3
4,3,7,256000,237,1470,4
5,5,7,256000,206,1933,6
6,10,2,192000,161,54881,1
12,134,7,256000,207,1126,5


In [0]:
g_index = pd.read_csv('fma_metadata/genres.csv')
g_index.drop(columns = ['#tracks','parent','top_level'])
g_index_lst = list(zip(g_index['genre_id'],g_index['title']))
g_index_lst

In [0]:
X1 = xtrain
y1= ytrain
# 84353,6 is too much for colab to run a logistic regression on.need to split again 
X_train, X_test, Y_train, Y_test = train_test_split(X1, y1, test_size=.8, random_state=42)

In [0]:
X = X_train
y= Y_train

In [89]:
log_reg = LogisticRegression().fit(X, y)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Index([nan, 'tracks', 'bit_rate', 'duration', 'interest', 'number'], dtype='object', name=1)

Index([nan, 'tracks', 'bit_rate', 'duration', 'interest', 'number'], dtype='object', name=1)

In [90]:
def sigmoid(x):
  return 1 / (1 + np.e**(-x))
print(log_reg.score(X,y))
print(log_reg.coef_)
log_reg.predict_proba(X)[0]


0.36917876815222833
[[-2.15227435e-05 -5.70286708e-02 -1.40854902e-05 -4.63691069e-03
   2.45247011e-05 -2.08123671e-02]
 [-5.14216914e-06  2.06670993e-08 -1.37411929e-05 -1.57306591e-07
   2.26442618e-05  1.35638940e-08]
 [-2.93278779e-06 -5.29214798e-02 -1.41857811e-05 -7.30182432e-03
   2.86785392e-06 -1.70765762e-02]
 [-1.42047232e-05 -9.73563263e-03 -2.62668775e-05 -5.52468690e-03
  -2.03006448e-04 -6.79594422e-03]
 [-7.74000022e-07 -2.51817988e-03 -3.96918303e-06 -9.56997678e-04
   8.82910087e-06 -7.23648206e-04]
 [-2.45806214e-06  6.20880242e-03 -3.36851703e-06  1.12631894e-03
  -5.15333287e-04  2.24214037e-03]
 [ 2.38157076e-06 -5.81414843e-03 -9.23521535e-06 -1.24724602e-03
  -1.11617472e-04 -2.93327802e-03]
 [-1.38470097e-06 -5.04702475e-03 -4.63084969e-06 -4.95729430e-03
  -2.21535798e-06 -1.06558599e-03]
 [ 5.61618027e-06 -3.36518474e-03 -1.05525181e-05 -3.61057141e-03
   1.60559752e-05 -1.69761613e-04]
 [-3.58417173e-06 -1.26156983e-02 -1.17647848e-05 -8.07879828e-04
   1.

array([0.00444411, 0.0376549 , 0.00392276, 0.00090666, 0.16397672,
       0.23271191, 0.06614495, 0.07948165, 0.04029095, 0.03849253,
       0.01699168, 0.01754122, 0.05222136, 0.22676612, 0.00467481,
       0.01377767])

With the features I used it appears the best predictors of genre would be Duration and Interest.  The coeficient for duration is the highest overall through all iterations, along withthe same trend for the interest feature. Track ID along with track amount per album had very little use in predictions. The most surprising thing for me would be the fact thatduration of the song has any effect in predicting the genre. 

This is the biggest data you've played with so far, and while it does generally fit in Colab, it can take awhile to run. That's part of the challenge!

Your tasks:
- Clean up the variable names in the dataframe
- Use logistic regression to fit a model predicting (primary/top) genre
- Inspect, iterate, and improve your model
- Answer the following questions (written, ~paragraph each):
  - What are the best predictors of genre?
  - What information isn't very useful for predicting genre?
  - What surprised you the most about your results?

*Important caveats*:
- This is going to be difficult data to work with - don't let the perfect be the enemy of the good!
- Be creative in cleaning it up - if the best way you know how to do it is download it locally and edit as a spreadsheet, that's OK!
- If the data size becomes problematic, consider sampling/subsetting
- You do not need perfect or complete results - just something plausible that runs, and that supports the reasoning in your written answers

If you find that fitting a model to classify *all* genres isn't very good, it's totally OK to limit to the most frequent genres, or perhaps trying to combine or cluster genres as a preprocessing step. Even then, there will be limits to how good a model can be with just this metadata - if you really want to train an effective genre classifier, you'll have to involve the other data (see stretch goals).

This is real data - there is no "one correct answer", so you can take this in a variety of directions. Just make sure to support your findings, and feel free to share them as well! This is meant to be practice for dealing with other "messy" data, a common task in data science.

## Resources and stretch goals

- Check out the other .csv files from the FMA dataset, and see if you can join them or otherwise fit interesting models with them
- [Logistic regression from scratch in numpy](https://blog.goodaudience.com/logistic-regression-from-scratch-in-numpy-5841c09e425f) - if you want to dig in a bit more to both the code and math (also takes a gradient descent approach, introducing the logistic loss function)
- Create a visualization to show predictions of your model - ideally show a confidence interval based on error!
- Check out and compare classification models from scikit-learn, such as [SVM](https://scikit-learn.org/stable/modules/svm.html#classification), [decision trees](https://scikit-learn.org/stable/modules/tree.html#classification), and [naive Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html). The underlying math will vary significantly, but the API (how you write the code) and interpretation will actually be fairly similar.
- Sign up for [Kaggle](https://kaggle.com), and find a competition to try logistic regression with
- (Not logistic regression related) If you enjoyed the assignment, you may want to read up on [music informatics](https://en.wikipedia.org/wiki/Music_informatics), which is how those audio features were actually calculated. The FMA includes the actual raw audio, so (while this is more of a longterm project than a stretch goal, and won't fit in Colab) if you'd like you can check those out and see what sort of deeper analysis you can do.